# 경사 하강법을 이용한 얕은 신경망 학습 

In [2]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [3]:
EPOCHS = 100

In [13]:
#네트워크 구조 정의
#입력 계층 : 2, 은닉계층 : 128(sigmoid activation), 출력계층 : 10 (softmax activation) 

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__() #상속클래스 이니셜라이즈 
        self.d1 = tf.keras.layers.Dense(128, input_dim = 2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
        
        
    
    #이니셜라이즈에서 어떤 레이어를 사용할건지 정의를 하면 call에서 실제 모델이 콜이 됬을때 
    #입력에서 출력까지 어떻게 연결할건지 정의
    def call(self, x, traninig = None, mask=None): 
        x = self.d1(x) 
        return self.d2(x)
        
        

## 학습 루프 

In [14]:
@tf.function # 이 어노테이션으로 인해서 최적화 할 수 있음 
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    
    #그라디언트를 계산해서 
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    train_loss(loss)
    train_metric(labels, predictions) #예측과 y값을 비교 

In [15]:
#데이터 셋 생성, 전처리 
np.random.seed(0)
pts = list() # 입력계층이 길이가 2이니까 스칼라 2
labels = list() #10개니까 0~9 

center_pts = np.random.uniform(-8.0, 8.0, (10,2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)
    
    
pts = np.stack(pts, axis=0).astype(np.float32)#gpu를 이용할 경우 float32로 해줘야 
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

In [16]:
#모델 생성
model = MyModel()

In [17]:
#손실 함수 및 최적화 알고리즘 설정
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [18]:
#평가 지표 
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [22]:
#학습 루프 
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template='Epoch {}, Loss:{}, Accuracy:{}'
    print(template.format(epoch +1, train_loss.result(), train_accuracy.result()*100))

Epoch 1, Loss:2.4702048301696777, Accuracy:10.0
Epoch 2, Loss:2.4701974391937256, Accuracy:10.0
Epoch 3, Loss:2.4701993465423584, Accuracy:10.0
Epoch 4, Loss:2.470200538635254, Accuracy:10.0
Epoch 5, Loss:2.4701900482177734, Accuracy:10.0
Epoch 6, Loss:2.47019100189209, Accuracy:10.0
Epoch 7, Loss:2.4701905250549316, Accuracy:10.0
Epoch 8, Loss:2.4701852798461914, Accuracy:10.0
Epoch 9, Loss:2.4701850414276123, Accuracy:10.0
Epoch 10, Loss:2.4701881408691406, Accuracy:10.0
Epoch 11, Loss:2.4701905250549316, Accuracy:10.0
Epoch 12, Loss:2.4701895713806152, Accuracy:10.0
Epoch 13, Loss:2.4701879024505615, Accuracy:10.0
Epoch 14, Loss:2.470186710357666, Accuracy:10.0
Epoch 15, Loss:2.470189332962036, Accuracy:10.0
Epoch 16, Loss:2.4701883792877197, Accuracy:10.0
Epoch 17, Loss:2.4701945781707764, Accuracy:10.0
Epoch 18, Loss:2.4701929092407227, Accuracy:10.0
Epoch 19, Loss:2.470189332962036, Accuracy:10.0
Epoch 20, Loss:2.4701921939849854, Accuracy:10.0
Epoch 21, Loss:2.4701881408691406, 

In [25]:
from tensorflow import keras
#keras Funtional API 사용 
def create_func_model():
	inputs = keras.Input(shape=(28,28))
	flatten = keras.layers.Flatten(input_shape=(28,28))(inputs)
	dense = keras.layers.Dense(128, activation='relu')(flatten)
	drop = keras.layers.Dropout(0.2)(dense)
	outputs = keras.layers.Dense(10, activation='softmax')(drop)
	model = keras.Model(inputs = inputs, outputs = outputs)
	return model 


In [26]:
fun_mo = create_func_model()

In [27]:
fun_mo.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [32]:
class Subclass(keras.Model):
	def __init__(self):
		super(Subclass, self).__init__()
		self.flatten = keras.layers.Flatten(input_shape=(28,28))
		self.dense = keras.layers.Dense(128, activation='relu')
		self.drop = keras.layers.Dropout(0.2)
		self.dense2 = keras.layers.Dense(10, activation='softmax')

	'''call 메소드에 training은 꼭 넣어줘야함.(False or None) 
		Dropout는 학습할때와 테스트할때가 다르기 때문에 학습인지 아닌지를 알려줘야함'''
	def call(self, x, training=False): 
		x = self.flatten(x)
		x = self.dense(x)
		x = self.drop(x)
		return self.dense2(x)

In [35]:
subc_model = Subclass()
inputs = np.zeros((1,28,28))
subc_model(inputs)
subc_model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "subclass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  100480    
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
____________________________

In [36]:
#가상의 data 만들어서 예측 
inputs = tf.random.normal((1,28,28))
outputs = subc_model(inputs)
pred = tf.argmax(outputs, -1)
print(f"Predicted class: {pred}")

Predicted class: [8]
